This scorecard contains multigig-enablement data at the State level, including:
Current XGSPON enablement - from LU QUAL
Current Multi-gig enablement - from LU QUAL (with hardcoded polygon id's)
Next month construction funded XGS-PON LUs - from PowerBI XGS Prep dashboard
Next month construction funded LUs with core network = Multigig

## Please do a fresh import for this report:
##### 1. Construction funding info to the end of the year from PowerBI - do not pull again until 2023
##### 2. Current copy of the summarized salesforce report  (because it's already been aggregated)

## Information needed from the engineers:
##### 1. Speedtest server ready?
##### 2. Out of MG & construction funded, how many LU's require additional 100G speedtest servers to support mg
##### 3. Final manual WC exclusions from mg opportunities list with sub info view

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

### Queries

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [3]:
# XGS qualifications count at wire center level
# 10/18/22 - add OLT_NET_ID to data pull

query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
            

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    ,SUBSTR(O.OLT_NET_ID,1,11) as OLT_NET_ID   --10/18/22 added per Joe advise
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  *
                    
            FROM    QD

                                """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
%%time

# count of MG enabled LUs at wire center level

mg_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in (
                    
                                        '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494','1189253', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306',-- MPLSMNBE 
                                          '1180028', '1178694', '1179569','1180333',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924','1179922', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237','1180468', -- CRYSMNCR
                                          '1180315','1189074','1189058', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507','1180076','1180079','1180492','1180497','1180486',  --TCSNAZMA
                                          '1180091','1179913','1189283',  --TCSNAZMA
                                          '1179641','1179914', --TCSNAZCR
                                          '1179840','1180355', --STTLWA04
                                          '1180086','1180111','1180327','1180436', --STTLWASU
                                          '1179576','1179672','1179971','1179977','1179979','1179981','1179987', --CLSPCOMA
                                          '1179988','1179989','1179994','1180005','1180006','1180041','1180123', --CLSPCOMA
                                          '1189130', --MLWKOR17
                                          '1179352','1180002','1189088', --MAGNUTNM
                                          '1179858','1180015','1180098','1180392','1180399', --SPKNWA01
                                          '1180027', --BRCTMNBC
                                          '1180491','1180509', --PHNXAZNO
                                          '1180488','1180496', --TCSNAZRN
                                          '1189200' --DNVRCOCL    
                                          
                                          ) --add in MG polygon IDs here
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 491 ms


In [5]:
# check data
print(mg_lu_ora.head())
print(df_ora.head())

  STATE PRIMARY_WIRE_CENTER_ID  MG_ENABLED_CNT
0    CO               CLSPCOMA            3393
1    MN               BRCTMNBC             363
2    CO               GLDNCOMA             410
3    OR               MLWKOR17              85
4    CO               DNVRCODC             166
  STATE PRIMARY_WIRE_CENTER_ID  XGS_ENABLED_CNT
0    CO               CLSPCOMA             3701
1    FL               CPCRFLXB            15013
2    MN               BRCTMNBC              363
3    CO               GLDNCOMA              410
4    CO               DNVRCOCP              320


In [6]:
# merge xgs enablement with mg enablement into one dataframe (mg enablement is a subset of xgs enablement)
luqual_enablement_df = df_ora.merge(mg_lu_ora, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID']).fillna(0)
luqual_enablement_df_save = luqual_enablement_df.copy() #export this as detail

In [7]:
# check data
luqual_enablement_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,XGS_ENABLED_CNT,MG_ENABLED_CNT
0,CO,CLSPCOMA,3701,3393.0
1,FL,CPCRFLXB,15013,0.0
2,MN,BRCTMNBC,363,363.0
3,CO,GLDNCOMA,410,410.0
4,CO,DNVRCOCP,320,0.0


### Merge LU QUAL data with MG Readiness worksheet

In [8]:
wc_mgr_df = pd.read_excel(r'Inputs/120122_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')
wc_mgr_df_save = wc_mgr_df.copy() # save for export in details

In [9]:
wc_mgr_df.columns

Index(['STATE', 'PRIMARY_WIRE_CENTER_ID', 'Opportunity Score', 'CBSA',
       'Core Status', 'Earliest Available For Multi-Gig', 'Cost for Multi-Gig',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '2022 Total', 'Jan', 'Feb', 'Mar',
       'Apr', 'May', 'Jun', 'July', 'Aug.1', 'Sep.1', 'Oct.1', 'Nov.1',
       'Dec.1', '2023 Total', 'Total'],
      dtype='object')

In [10]:
# obtain only relevant columns
# 10/7/22 - added opportunity score to the data pull
wc_mgr_df = wc_mgr_df[['STATE','PRIMARY_WIRE_CENTER_ID','Core Status','Opportunity Score']]

In [11]:
# clean data - rename columns
wc_mgr_df = wc_mgr_df.rename(columns = {'Core Status': 'CORE_STATUS',
                                        'Opportunity Score': 'OPP_SCORE'})

In [12]:
# merge mg readiness data with lu qual xgs enablement for wc master list
print(wc_mgr_df.head())
print(luqual_enablement_df.head())

xgs_wc_master_df = wc_mgr_df.merge(luqual_enablement_df, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

  STATE PRIMARY_WIRE_CENTER_ID    CORE_STATUS  OPP_SCORE
0    UT               STGRUTMA       MultiGIG  12.040135
1    CO               LTTNCOMA  XGS-PON - 180  10.510281
2    FL               STCDFLXA      940M Only   8.930216
3    UT               GTVLUTMA       MultiGIG  18.998338
4    ID               BOISIDMA       MultiGIG  17.283587
  STATE PRIMARY_WIRE_CENTER_ID  XGS_ENABLED_CNT  MG_ENABLED_CNT
0    CO               CLSPCOMA             3701          3393.0
1    FL               CPCRFLXB            15013             0.0
2    MN               BRCTMNBC              363           363.0
3    CO               GLDNCOMA              410           410.0
4    CO               DNVRCOCP              320             0.0


In [13]:
# set core status of wire centers with actual MG enablement to MultiGIG, regardless of worksheet status
xgs_wc_master_df.loc[xgs_wc_master_df['MG_ENABLED_CNT']>0,'CORE_STATUS'] = 'MultiGIG'

In [14]:
# base df containing xgs enablement counts, mg enablement counts, and the core network status at wire center level
print(xgs_wc_master_df.shape) # this should pull more wire centers that are not on mg readiness worksheet but in lu_qual_o
xgs_wc_master_df_save = xgs_wc_master_df.copy() #save a copy for export

(218, 6)


### Getting state-level construction funding view

In [15]:
# xgs_wc_master_df as base table = xgs_wc_master_df

# funding information is also at wire center level, join at wire center level then aggregate up

In [16]:
#Import Funding data - export most recent file from PowerBI
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/Details_Export_Table.xlsx',sheet_name = 'Export')
Construction_Funding_df_save = Construction_Funding_df.copy() # saving a copy for export later

In [17]:
Construction_Funding_df = Construction_Funding_df[['STATE','PRIMARY_WIRE_CENTER_ID','PHASE_ECD_LOGIC','PHASE_NUM_OF_LUS']]
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].astype(str)

In [18]:
Construction_pivot = Construction_Funding_df.groupby(by = ["STATE","PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index=['STATE','PRIMARY_WIRE_CENTER_ID'], columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()

In [19]:
# check data
Construction_pivot = Construction_pivot.fillna(0)
Construction_pivot_save = Construction_pivot.copy() # saving this for export later
Construction_pivot.head()

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022-09,2022-10,2022-11,2022-12
0,AZ,TCSNAZFW,0.0,0.0,0.0,463.0
1,AZ,TCSNAZMA,0.0,0.0,840.0,2188.0
2,AZ,TCSNAZRN,0.0,0.0,561.0,0.0
3,AZ,TCSNAZSW,0.0,0.0,0.0,249.0
4,CO,CLSPCOMA,0.0,0.0,0.0,985.0


In [20]:
# clean data - manually adjust field names
Construction_pivot = Construction_pivot.rename(columns = {
                                     '2022-09': '2022SEP_CONSTRUCTION_FUNDED',
                                     '2022-10': '2022OCT_CONSTRUCTION_FUNDED',
                                     '2022-11': '2022NOV_CONSTRUCTION_FUNDED',
                                     '2022-12': '2022DEC_CONSTRUCTION_FUNDED'})

In [21]:
# check data
Construction_pivot[(Construction_pivot['STATE']=='WA')&(Construction_pivot['2022DEC_CONSTRUCTION_FUNDED']>0)]

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
35,WA,FDWYWA01,0.0,0.0,0.0,168.0
36,WA,FRKSWAXA,0.0,0.0,0.0,201.0
37,WA,SPKNWA01,134.0,0.0,0.0,1125.0
38,WA,SPKNWAWH,0.0,0.0,0.0,208.0
39,WA,STTLWA04,214.0,88.0,0.0,84.0
41,WA,STTLWASU,168.0,262.0,0.0,346.0
42,WA,VANCWA01,0.0,0.0,0.0,250.0


In [22]:
# join construction funding to base table
Funded_XGS_WC = xgs_wc_master_df.merge(Construction_pivot, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

In [23]:
Funded_XGS_WC[(Funded_XGS_WC["STATE"]=='WA') & (Funded_XGS_WC["2022DEC_CONSTRUCTION_FUNDED"]>0)]

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
33,WA,VANCWA01,MultiGIG,12.981307,NaN,NaN,0.0,0.0,0.0,250.0
42,WA,STTLWASU,MultiGIG,11.683616,492.0,492.0,168.0,262.0,0.0,346.0
55,WA,STTLWA04,MultiGIG,10.624742,298.0,298.0,214.0,88.0,0.0,84.0
76,WA,SPKNWA01,MultiGIG,9.496253,950.0,950.0,134.0,0.0,0.0,1125.0
84,WA,FDWYWA01,MultiGIG,8.803858,NaN,NaN,0.0,0.0,0.0,168.0
94,WA,SPKNWAWH,MultiGIG,8.260053,NaN,NaN,0.0,0.0,0.0,208.0
212,WA,FRKSWAXA,New to view,NaN,NaN,NaN,0.0,0.0,0.0,201.0


#### View prep

In [24]:
# Creating view

Funded_State_view = Funded_XGS_WC.drop(['OPP_SCORE'], axis = 1).groupby(['STATE'], as_index = False).sum()

# add grand total row - adjust names manually if needed
Funded_State_view.loc[len(Funded_State_view.index)] = ['Grand Total',
                                                       Funded_State_view['XGS_ENABLED_CNT'].sum(),
                                                       Funded_State_view['MG_ENABLED_CNT'].sum(),
                                                       Funded_State_view['2022SEP_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022OCT_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022NOV_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022DEC_CONSTRUCTION_FUNDED'].sum()]

In [25]:
# check data
Funded_State_view.head()

,STATE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,16987.0,14316.0,0.0,0.0,1401.0,2900.0
1,CO,9276.0,4920.0,0.0,0.0,430.0,13030.0
2,FL,27539.0,0.0,0.0,0.0,5286.0,3017.0
3,IA,0.0,0.0,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# view ready for export 

# adjust names manually if needed
Funded_State_view = Funded_State_view[['STATE','2022SEP_CONSTRUCTION_FUNDED','2022OCT_CONSTRUCTION_FUNDED','2022NOV_CONSTRUCTION_FUNDED','2022DEC_CONSTRUCTION_FUNDED']]
Funded_State_view

,STATE,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,0.0,0.0,1401.0,2900.0
1,CO,0.0,0.0,430.0,13030.0
2,FL,0.0,0.0,5286.0,3017.0
3,IA,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0
5,MN,1130.0,745.0,1629.0,3587.0
6,NE,0.0,0.0,0.0,0.0
7,NV,0.0,0.0,0.0,135.0
8,OR,76.0,0.0,85.0,0.0
9,UT,36.0,68.0,906.0,3911.0


### Getting multigig opportunity summary view

In [27]:
# adjust names manually if needed
MG_Opportunity_Funding = Funded_XGS_WC[['STATE','PRIMARY_WIRE_CENTER_ID','XGS_ENABLED_CNT','MG_ENABLED_CNT','2022DEC_CONSTRUCTION_FUNDED','CORE_STATUS','OPP_SCORE']]

# adding new column that will get construction funding only if the core status is MultiGIG
# 10/31/22 - exclude denver cocp wire center site from multigig-candidate
# 11/30/22 - exclude STGRUTMA,FDWYWA01 from multigig-candidate because of latency

MG_Opportunity_Funding['2022DEC_CONSTRUCTION_FUNDED_MG'] = np.where(
        (MG_Opportunity_Funding['CORE_STATUS'] == 'MultiGIG')
        & (MG_Opportunity_Funding['PRIMARY_WIRE_CENTER_ID'] != "DNVRCOCP") 
        & (MG_Opportunity_Funding['PRIMARY_WIRE_CENTER_ID'] != "STGRUTMA")
        & (MG_Opportunity_Funding['PRIMARY_WIRE_CENTER_ID'] != "VANCWA01"),
        MG_Opportunity_Funding['2022DEC_CONSTRUCTION_FUNDED'],0)

C:\Windows\Temp\ipykernel_6300\1136068132.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MG_Opportunity_Funding['2022DEC_CONSTRUCTION_FUNDED_MG'] = np.where(


In [28]:
# check data

MG_Opportunity_Funding.head()

,STATE,PRIMARY_WIRE_CENTER_ID,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022DEC_CONSTRUCTION_FUNDED,CORE_STATUS,OPP_SCORE,2022DEC_CONSTRUCTION_FUNDED_MG
0,UT,STGRUTMA,3136.0,0.0,1580.0,MultiGIG,12.040135,0.0
1,CO,LTTNCOMA,264.0,0.0,NaN,XGS-PON - 180,10.510281,0.0
2,FL,STCDFLXA,NaN,NaN,579.0,940M Only,8.930216,0.0
3,UT,GTVLUTMA,NaN,NaN,NaN,MultiGIG,18.998338,NaN
4,ID,BOISIDMA,NaN,NaN,NaN,MultiGIG,17.283587,NaN


In [29]:
MG_Opportunity_Funding_WC_save = MG_Opportunity_Funding.copy() # save copy for export

#### Prep view

In [30]:
# Creating view
MG_Opportunity_Funding_State_view = MG_Opportunity_Funding.groupby(['STATE'], as_index = False)[['XGS_ENABLED_CNT','MG_ENABLED_CNT','2022DEC_CONSTRUCTION_FUNDED','2022DEC_CONSTRUCTION_FUNDED_MG']].sum()

# add grand total row - manually adjust the field names
MG_Opportunity_Funding_State_view.loc[len(MG_Opportunity_Funding_State_view.index)] = ['Grand Total',
                                                       MG_Opportunity_Funding_State_view['XGS_ENABLED_CNT'].sum(),
                                                       MG_Opportunity_Funding_State_view['MG_ENABLED_CNT'].sum(),
                                                       MG_Opportunity_Funding_State_view['2022DEC_CONSTRUCTION_FUNDED'].sum(),
                                                       MG_Opportunity_Funding_State_view['2022DEC_CONSTRUCTION_FUNDED_MG'].sum()]

In [31]:
# check data

MG_Opportunity_Funding_State_view # view ready for export

,STATE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022DEC_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED_MG
0,AZ,16987.0,14316.0,2900.0,2437.0
1,CO,9276.0,4920.0,13030.0,5748.0
2,FL,27539.0,0.0,3017.0,0.0
3,IA,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0
5,MN,10508.0,9711.0,3587.0,3587.0
6,NE,0.0,0.0,0.0,0.0
7,NV,94.0,0.0,135.0,0.0
8,OR,349.0,349.0,0.0,0.0
9,UT,4060.0,659.0,3911.0,1983.0


### Getting Subs views

Getting current MG opportunity

In [32]:
# create original view but with all MG wire centers from above base
# active subs y/n -- pending subs y/n -- earliest due date for pending subs

# create sub view of above with only wire centers in states that have at least one mg enablement
# graph to illustrate MG opportunity

In [33]:
# start with same base - check data
xgs_wc_master_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT
0,UT,STGRUTMA,MultiGIG,12.040135,3136.0,0.0
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0
2,FL,STCDFLXA,940M Only,8.930216,NaN,NaN
3,UT,GTVLUTMA,MultiGIG,18.998338,NaN,NaN
4,ID,BOISIDMA,MultiGIG,17.283587,NaN,NaN


In [34]:
# fillna

xgs_wc_master_df[['XGS_ENABLED_CNT','MG_ENABLED_CNT']] = xgs_wc_master_df[['XGS_ENABLED_CNT','MG_ENABLED_CNT']].fillna(0)

In [35]:
# derive MG untapped column
xgs_wc_master_df['MG_UNTAPPED'] = xgs_wc_master_df['XGS_ENABLED_CNT'] - xgs_wc_master_df['MG_ENABLED_CNT']

In [36]:
# check data
xgs_wc_master_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED
0,UT,STGRUTMA,MultiGIG,12.040135,3136.0,0.0,3136.0
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0,264.0
2,FL,STCDFLXA,940M Only,8.930216,0.0,0.0,0.0
3,UT,GTVLUTMA,MultiGIG,18.998338,0.0,0.0,0.0
4,ID,BOISIDMA,MultiGIG,17.283587,0.0,0.0,0.0


In [37]:
# import salesforce summary to get active/pending subs
sf_df_sum = pd.read_excel(r'Salesforce Data/Salesforce_Summary.xlsx',sheet_name = 'Summary')
sf_df_details = pd.read_excel(r'Salesforce Data/Salesforce_Summary.xlsx',sheet_name = 'Details') 

In [38]:
# get relevant columns in salesforce summary df
sf_df_sum = sf_df_sum[['WIRE_CENTER_ID','Total Active Subscribers','Total Pending Activation Subscribers']].rename(columns = {'WIRE_CENTER_ID':'PRIMARY_WIRE_CENTER_ID'})

In [39]:
# wire centers with active subs
sf_df_sum = sf_df_sum[(sf_df_sum['Total Active Subscribers']>0)|(sf_df_sum['Total Pending Activation Subscribers']>0)]

In [40]:
# check data
sf_df_sum

,PRIMARY_WIRE_CENTER_ID,Total Active Subscribers,Total Pending Activation Subscribers
0,0,0,70
399,PHNXAZNE,12,1
400,PHNXAZNO,1,8
401,PHNXAZSO,45,1
402,PHNXAZWE,0,4
403,SPRSAZEA,35,1
404,SPRSAZMA,17,2
405,TCSNAZCR,21,0
406,TCSNAZFW,10,0
407,TCSNAZMA,138,16


In [41]:
# get earliest pending activation due date

from datetime import timedelta, date
from pandas.tseries.offsets import DateOffset

yesterday_date = date.today() - timedelta(days=90)
future_date = date.today() + timedelta(days=90)

duedate_mask = (sf_df_details['Account Status']=='Pending Activation') & (pd.to_datetime(sf_df_details['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(sf_df_details['Due Date']).dt.date <= future_date)
sf_df_duedates = sf_df_details.loc[duedate_mask,['Wire Center','Due Date']]

In [42]:
# update Due Date field to date only
sf_df_duedates['Due Date'] = pd.to_datetime(sf_df_duedates['Due Date']).dt.date

In [43]:
# get earliest due date by wire center

sf_df_duedates = sf_df_duedates.groupby(['Wire Center'], as_index = False).agg(np.min).rename(columns = {'Wire Center':'PRIMARY_WIRE_CENTER_ID',
                                                                                                         'Due Date': 'Earliest Due Date'})

In [44]:
# join due dates to summary

sf_df_sum = sf_df_sum.merge(sf_df_duedates, on = 'PRIMARY_WIRE_CENTER_ID', how = 'left')

In [45]:
# join salesforce to xgs_wc_master_df

subs_view = xgs_wc_master_df.merge(sf_df_sum, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

#### Prep view

In [46]:
# check data
subs_view.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Total Active Subscribers,Total Pending Activation Subscribers,Earliest Due Date
0,UT,STGRUTMA,MultiGIG,12.040135,3136.0,0.0,3136.0,55.0,23.0,2022-12-16
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0,264.0,5.0,0.0,NaN
2,FL,STCDFLXA,940M Only,8.930216,0.0,0.0,0.0,NaN,NaN,NaN
3,UT,GTVLUTMA,MultiGIG,18.998338,0.0,0.0,0.0,NaN,NaN,NaN
4,ID,BOISIDMA,MultiGIG,17.283587,0.0,0.0,0.0,NaN,NaN,NaN


In [47]:
# update subs columns to Y/N

subs_view['Active Subs Y/N'] = np.where(subs_view['Total Active Subscribers'] > 0, 'Y', 'N')
subs_view['Pending Subs Y/N'] = np.where(subs_view['Total Pending Activation Subscribers'] > 0, 'Y', 'N')

In [48]:
subs_view_save = subs_view.copy() # save copy for export

In [49]:
# drop columns
subs_view = subs_view.drop(['Total Active Subscribers','Total Pending Activation Subscribers'], axis = 1)

In [50]:
# reorder columns

subs_view = subs_view[['STATE','PRIMARY_WIRE_CENTER_ID','CORE_STATUS','OPP_SCORE','XGS_ENABLED_CNT','MG_ENABLED_CNT','MG_UNTAPPED','Active Subs Y/N','Pending Subs Y/N','Earliest Due Date']]

In [51]:
# only wire centers with XGS enablement

subs_view = subs_view[subs_view['XGS_ENABLED_CNT'] > 0]

In [52]:
# ready for export

subs_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
0,UT,STGRUTMA,MultiGIG,12.040135,3136.0,0.0,3136.0,Y,Y,2022-12-16
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0,264.0,Y,N,NaN
8,CO,FONTCOMA,MultiGIG,15.995734,246.0,0.0,246.0,N,N,NaN
10,UT,MAGNUTNM,MultiGIG,15.255279,314.0,314.0,0.0,Y,Y,2022-12-23
12,CO,DNVRCOCP,MultiGIG,14.734274,320.0,0.0,320.0,N,N,NaN
14,CO,SCRTCOMA,MultiGIG,14.355833,94.0,0.0,94.0,Y,N,NaN
19,MN,BRCTMNBC,MultiGIG,14.039748,363.0,363.0,0.0,N,N,NaN
22,CO,DNVRCOMA,MultiGIG,13.735440,439.0,439.0,0.0,Y,Y,2023-01-02
23,MN,MPLSMNGE,MultiGIG,13.691429,1772.0,1772.0,0.0,Y,Y,2022-12-12
27,CO,CLSPCOMA,MultiGIG,13.343808,3701.0,3393.0,308.0,Y,Y,2022-12-22


### MG only subs view

In [53]:
# start with subs view
mg_subs_view = subs_view.copy()

In [54]:
# getting only Core Network Status MultiGIG
mg_subs_view = mg_subs_view[mg_subs_view['CORE_STATUS'] == 'MultiGIG'].sort_values(by = ['STATE','PRIMARY_WIRE_CENTER_ID'])

In [55]:
# check data
mg_subs_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
35,AZ,PHNXAZNE,MultiGIG,12.720800,210.0,210.0,0.0,Y,Y,2022-12-21
133,AZ,PHNXAZNO,MultiGIG,11.680925,243.0,243.0,0.0,Y,Y,2022-12-21
48,AZ,PHNXAZSO,MultiGIG,11.211576,1624.0,1624.0,0.0,Y,Y,2022-12-13
92,AZ,TCSNAZCR,MultiGIG,8.356103,400.0,400.0,0.0,Y,N,NaN
101,AZ,TCSNAZMA,MultiGIG,7.669243,11522.0,11278.0,244.0,Y,Y,2022-12-14
100,AZ,TCSNAZRN,MultiGIG,7.889057,1192.0,561.0,631.0,Y,Y,2022-12-26
83,AZ,TCSNAZSW,MultiGIG,8.926796,445.0,0.0,445.0,N,N,NaN
53,CO,ARVDCOMA,MultiGIG,10.753984,273.0,196.0,77.0,Y,Y,2022-12-21
27,CO,CLSPCOMA,MultiGIG,13.343808,3701.0,3393.0,308.0,Y,Y,2022-12-22
64,CO,CLSPCOPV,MultiGIG,10.183161,2440.0,0.0,2440.0,Y,Y,2022-12-13


In [56]:
# get data with non-zero untapped MG

# 10/31/22 - exclude denver cocp wire center site from multigig-candidate
# 11/30/22 - exclude STGRUTMA,FDWYWA01 from multigig-candidate because of latency

mask = (mg_subs_view['MG_UNTAPPED'] > 0) \
        & (mg_subs_view['PRIMARY_WIRE_CENTER_ID'] != "DNVRCOCP") \
        & (mg_subs_view['PRIMARY_WIRE_CENTER_ID'] != "STGRUTMA") \
        & (mg_subs_view['PRIMARY_WIRE_CENTER_ID'] != "VANCWA01")

mg_subs_untapped_view = mg_subs_view[mask]

In [57]:
# ready for export

mg_subs_untapped_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
101,AZ,TCSNAZMA,MultiGIG,7.669243,11522.0,11278.0,244.0,Y,Y,2022-12-14
100,AZ,TCSNAZRN,MultiGIG,7.889057,1192.0,561.0,631.0,Y,Y,2022-12-26
83,AZ,TCSNAZSW,MultiGIG,8.926796,445.0,0.0,445.0,N,N,NaN
53,CO,ARVDCOMA,MultiGIG,10.753984,273.0,196.0,77.0,Y,Y,2022-12-21
27,CO,CLSPCOMA,MultiGIG,13.343808,3701.0,3393.0,308.0,Y,Y,2022-12-22
64,CO,CLSPCOPV,MultiGIG,10.183161,2440.0,0.0,2440.0,Y,Y,2022-12-13
153,CO,DNVRCOSW,MultiGIG,9.923813,70.0,0.0,70.0,Y,N,NaN
8,CO,FONTCOMA,MultiGIG,15.995734,246.0,0.0,246.0,N,N,NaN
14,CO,SCRTCOMA,MultiGIG,14.355833,94.0,0.0,94.0,Y,N,NaN
29,MN,CRYSMNCR,MultiGIG,13.288129,1777.0,1275.0,502.0,Y,Y,2022-12-20


### New tab: Internal document - polygon info

In [58]:
# exclude existing polygons that are mg enabled. Get opportunity scores

In [59]:
# using mg_subs_untapped_view as base, create polygon info sheet
mg_info_view = mg_subs_untapped_view.copy()

In [60]:
# get relevant polygon info at wire center level
polygon_info = df_ora_details[['PRIMARY_WIRE_CENTER_ID','UNI_NDSJOBNUMBER','UNI_POLYGONID', 'OLT_NET_ID']]

In [61]:
# keep unique values
polygon_info = polygon_info.drop_duplicates(ignore_index=True)

In [62]:
# set OLT_NET_ID NaN to "none"
polygon_info['OLT_NET_ID'] = polygon_info['OLT_NET_ID'].fillna("")

In [63]:
polygon_info = polygon_info.groupby(['PRIMARY_WIRE_CENTER_ID','UNI_POLYGONID']) \
   .agg({'UNI_NDSJOBNUMBER' : lambda x: ','.join(x.unique()),
         'OLT_NET_ID' : ','.join}) \
   .reset_index() \
   .reindex(columns=polygon_info.columns)

In [64]:
# check data
polygon_info[polygon_info['PRIMARY_WIRE_CENTER_ID']=='MAGNUTNM']

,PRIMARY_WIRE_CENTER_ID,UNI_NDSJOBNUMBER,UNI_POLYGONID,OLT_NET_ID
89,MAGNUTNM,N.882095,1179352,",MAGNUTNMOL4"
90,MAGNUTNM,N.882094,1180002,",MAGNUTNMOL4"
91,MAGNUTNM,N.895257,1189088,


In [65]:
# join polygon info to mg view

mg_info_view = mg_info_view[['STATE','PRIMARY_WIRE_CENTER_ID','OPP_SCORE']].merge(polygon_info, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [66]:
# join to salesforce data at polygon id level
# show only salesforce data that has active subscribers and pending subscribers within 3 months in future
duedate_mask = (sf_df_details['Account Status']=='Pending Activation') & (pd.to_datetime(sf_df_details['Due Date']).dt.date >= date.today()) & (pd.to_datetime(sf_df_details['Due Date']).dt.date <= future_date)
sf_df_details_polygon = sf_df_details[(sf_df_details['Account Status'] == 'Active') | duedate_mask]

In [67]:
sf_df_details_polygon = sf_df_details_polygon[['Polygon ID','Account Status']].groupby(['Polygon ID','Account Status'], as_index = False).size().pivot(index = 'Polygon ID', columns = 'Account Status', values = 'size').reset_index().fillna(0)


In [68]:
sf_df_details_polygon[sf_df_details_polygon['Polygon ID']==1179352]

Account Status,Polygon ID,Active,Pending Activation
30,1179352,1.0,0.0


In [69]:
# update salesforce polygon column to object
sf_df_details_polygon['Polygon ID'] = sf_df_details_polygon['Polygon ID'].astype(str)

In [70]:
mg_info_view = mg_info_view.rename(columns = {'UNI_POLYGONID': 'Polygon ID'})
mg_info_view = mg_info_view.merge(sf_df_details_polygon[['Polygon ID','Active','Pending Activation']], on = 'Polygon ID', how = 'left').fillna(0)

In [71]:
mg_info_view

,STATE,PRIMARY_WIRE_CENTER_ID,OPP_SCORE,UNI_NDSJOBNUMBER,Polygon ID,OLT_NET_ID,Active,Pending Activation
0,AZ,TCSNAZMA,7.669243,N.957868,1179639,",TCSNAZMAOL6",26.0,0.0
1,AZ,TCSNAZMA,7.669243,N.957886,1179689,",TCSNAZMAOL5",15.0,0.0
2,AZ,TCSNAZMA,7.669243,N.916206,1179690,",TCSNAZMAOL7",13.0,3.0
3,AZ,TCSNAZMA,7.669243,N.957879,1179700,",TCSNAZMAOL7",9.0,0.0
4,AZ,TCSNAZMA,7.669243,N.927853,1179701,",TCSNAZMAOL6",4.0,3.0
...,...,...,...,...,...,...,...,...
71,MN,CRYSMNCR,13.288129,N.916148,1180237,",CRYSMNCROL5",1.0,0.0
72,MN,CRYSMNCR,13.288129,N.916084,1180265,",CRYSMNCROL4",2.0,0.0
73,MN,CRYSMNCR,13.288129,N.916145,1180286,,0.0,1.0
74,MN,CRYSMNCR,13.288129,N.916093,1180468,",CRYSMNCROL7",0.0,2.0


In [72]:
# reorder columns, rename columns
mg_info_view = mg_info_view[['STATE','PRIMARY_WIRE_CENTER_ID','Polygon ID','UNI_NDSJOBNUMBER','OLT_NET_ID','Active','Pending Activation']]

In [73]:
mg_info_view = mg_info_view.rename(columns = {'Polygon ID':'POLYGON_ID',
                               'Active': 'ACTIVE_SUBS_CNT',
                               'Pending Activation': 'PENDING_SUBS_CNT'})

mg_info_view.head()

,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,UNI_NDSJOBNUMBER,OLT_NET_ID,ACTIVE_SUBS_CNT,PENDING_SUBS_CNT
0,AZ,TCSNAZMA,1179639,N.957868,",TCSNAZMAOL6",26.0,0.0
1,AZ,TCSNAZMA,1179689,N.957886,",TCSNAZMAOL5",15.0,0.0
2,AZ,TCSNAZMA,1179690,N.916206,",TCSNAZMAOL7",13.0,3.0
3,AZ,TCSNAZMA,1179700,N.957879,",TCSNAZMAOL7",9.0,0.0
4,AZ,TCSNAZMA,1179701,N.927853,",TCSNAZMAOL6",4.0,3.0


In [74]:
# join in xgs enablement and mg enablement info

# XGS qualifications count at polygon level

polygon_xgs_query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,count(distinct LUID) as xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
            

                                """


polygon_xgs_enablement_df = pd.read_sql(polygon_xgs_query, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [75]:
# mg enablement at polygon level
polygon_multigig_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in (
                                          '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494','1189253', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306',-- MPLSMNBE 
                                          '1180028', '1178694', '1179569','1180333',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924','1179922', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237','1180468', -- CRYSMNCR
                                          '1180315','1189074','1189058', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507','1180076','1180079','1180492','1180497','1180486',  --TCSNAZMA
                                          '1180091','1179913','1189283',  --TCSNAZMA
                                          '1179641','1179914', --TCSNAZCR
                                          '1179840','1180355', --STTLWA04
                                          '1180086','1180111','1180327','1180436', --STTLWASU
                                          '1179576','1179672','1179971','1179977','1179979','1179981','1179987', --CLSPCOMA
                                          '1179988','1179989','1179994','1180005','1180006','1180041','1180123', --CLSPCOMA
                                          '1189130', --MLWKOR17
                                          '1179352','1180002','1189088', --MAGNUTNM
                                          '1179858','1180015','1180098','1180392','1180399', --SPKNWA01
                                          '1180027', --BRCTMNBC
                                          '1180491','1180509', --PHNXAZNO
                                          '1180488','1180496', --TCSNAZRN
                                          '1189200' --DNVRCOCL    
                                          ) --add in MG polygon IDs here
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,count(distinct LUID) as mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID


"""

polygon_multigig_enablement_df = pd.read_sql(polygon_multigig_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [76]:
# check data
print(polygon_multigig_enablement_df.head())
print(polygon_xgs_enablement_df.head())

  STATE PRIMARY_WIRE_CENTER_ID UNI_POLYGONID  MG_ENABLED_CNT
0    MN               MPLSMNTF       1179157             365
1    CO               CLSPCOMA       1179971             150
2    CO               GLDNCOMA       1179674             155
3    MN               CRYSMNCR       1180265             140
4    CO               DNVRCOCL       1189200             316
  STATE PRIMARY_WIRE_CENTER_ID UNI_POLYGONID  XGS_ENABLED_CNT
0    FL               CPCRFLXB       1179775              448
1    FL               CPCRFLXB       1180058              622
2    FL               CPCRFLXB       1180059              560
3    CO               CLSPCOMA       1179971              150
4    CO               CLSPCOPV       1180262              184


In [77]:
polygon_enablement_mg_xgs = polygon_xgs_enablement_df.merge(polygon_multigig_enablement_df, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID','UNI_POLYGONID']).fillna(0)

In [78]:
polygon_enablement_mg_xgs = polygon_enablement_mg_xgs.rename(columns = {'UNI_POLYGONID': 'POLYGON_ID'})

In [79]:
mg_info_view = mg_info_view.merge(polygon_enablement_mg_xgs, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID'])

In [80]:
mg_info_view['MG_UNTAPPED_LU'] = mg_info_view['XGS_ENABLED_CNT'] - mg_info_view['MG_ENABLED_CNT']

In [81]:
# export

import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_Multigig_Enablement_Scorecard_Report.xlsx"

with pd.ExcelWriter(excelfilename) as writer:
    mg_subs_untapped_view.to_excel(writer, sheet_name='MG_Subs_Untapped_Summary', index = False)
    subs_view.to_excel(writer, sheet_name='XGS_Subs_Summary', index = False)
    MG_Opportunity_Funding_State_view.to_excel(writer, sheet_name='MG_Enablement_Tracker', index = False)
    Funded_State_view.to_excel(writer, sheet_name='Construction_Funded_Outlook', index = False)
    mg_info_view.to_excel(writer, sheet_name='Polygon_info', index = False)
 
    
    # raw data
    df_ora_details.to_excel(writer, sheet_name='LUQUALO_Details', index = False)
    Construction_Funding_df_save.to_excel(writer, sheet_name='XGSPrep_Details_PowerBI', index = False)
    wc_mgr_df_save.to_excel(writer, sheet_name='MG_Readiness_details', index = False)
    


In [82]:
#create ad-hoc directory
directory = "Output/" + TodaysDate + " ad-hoc views"
os.makedirs(directory)